# Text Sentiment Analysis

1. Importing Tools & Libraries

In [1]:
import pandas as pd
import numpy as np
import re
import csv
import nltk
import joblib
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

2. Download Required NLTK Resources

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to C:\Users\Al hafiz
[nltk_data]     computers\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Al hafiz
[nltk_data]     computers\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Al hafiz
[nltk_data]     computers\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Al hafiz
[nltk_data]     computers\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

3. Load The Dataset

In [3]:
df=pd.read_csv("IMDB Dataset.csv" , 
               encoding = 'utf-8' ,
               on_bad_lines = 'skip',
               quoting=csv.QUOTE_NONE,)
print("CSV loaded successfully")


CSV loaded successfully


4. Removing NaN Values

In [4]:
df=df.dropna(subset=['sentiment'])
df.head()

,,,,,,,,,,,,,,,,,,,,,,,,,,review,sentiment
"""One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right",as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence,which set in right from the word GO. Trust me,this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs,sex or violence. Its is hardcore,in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City,an experimental section of the prison where all the cells have glass fronts and face inwards,so privacy is not high on the agenda. Em City is home to many..Aryans,Muslims,gangstas,Latinos,Christians,Italians,Irish and more....so scuffles,death stares,dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences,forget charm,forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal,I couldn't say I was ready for it,but as I watched more,I developed a taste for Oz,and got accustomed to the high levels of graphic violence. Not just violence,but injustice (crooked guards who'll be sold out for a nickel,inmates who'll kill on order and get away with it,well mannered,middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz,you may become comfortable with what is uncom...,positive
"""Gamers: DR is not a fancy made movie",it's more like amateur video. Horrible magic effect,really fake fireball,terribly made dungeon,castle,village...... sword,axe,shield,robe,plate..... okok... everything. You will need about 10 minutes to adjust your expecting on visual,then you will get 105 minutes of fun.<br /><br />I'm from Hongkong and it's really hard to find RPers,none of my friends play RPG and I always fancy to be one of the character in the world of D&D. Watching Gamers: DR just show me what would it be like to be a gamer. You see rule books,dices,game set,"etc etc etc; You hear terms like """"fighter""""","""""wizard""""","""""hit point""""","""""level""""","""""character""""","""""flaming hand""""","""""Chaotic Evil"""".<br /><br />What RPG fancy me is that it let you do anything u want to",not bonded by software RPG. Gamers: DR provide the same element,you wont know what happen next and it probably just make you laugh to dead. The movie goes both gamers's real life as well as in the D&D world. You will hear the gamer cast the dice when the character in game take action,which make you feel you really participle in the game.<br /><br />I don't want to spoil anything,but in short,Gamers: DR is a must watch movie for RPG lovers. For people never play RPG game,"I'm sure you still get many fun from it.""",positive
"""First",let me just comment on what I liked about the movie. The special effects were fantastic,and very rarely did I feel like I was watching a video game. There,that is the last nice thing I have to say about this film. In fact,I would just like everyone reading this to take note that I can't even put into words how hard it was for me to write this review without swearing. <br /><br />I have innumerable complaints about the film,"but four major complaints jump to mind. My first major complaint has to do with the incredible cheesiness of the """"plot twist"""" (if you can call it that since most people probably saw it coming a mile away) where Lois's 5 year-old son turns out to be the super-powered child of Superman. When the crying super-child throws a piano at Lex's henchman to save his mother",I almost got up and left the theater. Singer could have made a much better Superman movie without resorting to cheap gi

5. Text Preprocessing Function

In [22]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    # 1. lowercase
    text = str(text).lower()
    # 2. Remove HTML Tags
    text = re.sub(r'<.*?>','',text)
    # 3. Remove non alphabetic characters
    text = re.sub(r'[^a-zA-Z]',' ',text)
    # 4. Tokenize
    words = nltk.word_tokenize(text)
    # 5. Remove Stopwords & Lemmatize
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    text=re.sub(r'[^a-z\s]','',text)
    text=re.sub(r'\s+','',text).strip()
    return' '.join(words)  

6. Apply Preprocessing

In [23]:
df['clean_review'] = df['review'].apply(preprocess_text)

7. Feature Engineering

In [24]:
vectorizer = TfidfVectorizer(max_features=5000)
x_train_tfidf=vectorizer.fit_transform(x_train)
x_test_tfidf=vectorizer.transform(x_test)
x = vectorizer.fit_transform(df['clean_review'])
y = df['sentiment'].map({'positive':1,'negative':0}) # Converts Labels to Binary

8. Train-Test Split

In [25]:
x=df['clean_review']
y=df['sentiment']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

9. Model Training

In [43]:
model=LogisticRegression()
model.fit(x_train_tfidf,y_train)

LogisticRegression()

10. Prediction & Evaluation

In [44]:
y_pred=model.predict(x_test_tfidf)
print(classification_report(y_test,y_pred,target_names=["negative","positive"]))

              precision    recall  f1-score   support

    negative       0.50      0.33      0.40        27
    positive       0.61      0.76      0.67        37

    accuracy                           0.58        64
   macro avg       0.55      0.55      0.54        64
weighted avg       0.56      0.58      0.56        64



11. Hperparameter Tuning

In [45]:
params={'C':[0.1,1,10],'penalty':['l2']}
grid=GridSearchCV(LogisticRegression(),param_grid=params,cv=5)
grid.fit(x_train_tfidf,y_train)
print(grid.best_params_)

{'C': 10, 'penalty': 'l2'}


In [46]:
pipeline_rf=Pipeline([
    ('tfidf',TfidfVectorizer(stop_words='english')),
    ('clf',RandomForestClassifier())
])
params_dist_rf={
    'tfidf__ngram_range':[(1,1),(1,2)],
    'tfidf__max_df':[0.5,0.75,1.0],
    'tfidf__min_df':[1,2,5],
    'clf__n_estimators':[100,200,300],
    'clf__max_depth':[None,10,20,30],
    'clf__min_samples_split':[2,5,10]
}
search_rf=RandomizedSearchCV(pipeline_rf,param_distributions=params_dist_rf,n_iter=20,cv=5,verbose=2,n_jobs=-1)
search_rf.fit(x_train,y_train)
print("Best Params (RF):",search_rf.best_params_)
print('Score:',search_rf.score(x_test,y_test))

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Params (RF): {'tfidf__ngram_range': (1, 1), 'tfidf__min_df': 5, 'tfidf__max_df': 1.0, 'clf__n_estimators': 100, 'clf__min_samples_split': 5, 'clf__max_depth': 20}
Score: 0.5625


In [47]:
y_pred=model.predict(x_test_tfidf)
print(classification_report(y_test,y_pred,target_names=["negative","positive"]))

              precision    recall  f1-score   support

    negative       0.50      0.33      0.40        27
    positive       0.61      0.76      0.67        37

    accuracy                           0.58        64
   macro avg       0.55      0.55      0.54        64
weighted avg       0.56      0.58      0.56        64



In [48]:
acc=accuracy_score(y_test,y_pred)
print(f"Accuracy:{acc:.2f}")

Accuracy:0.58
